In [6]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/europeanpatentofficev2/en-up-guidelines-2025-pre-publication.pdf
/kaggle/input/qst-and-answer-oeb/Questions Sup OEB.docx


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qst-and-answer-oeb/Questions Sup OEB.docx
/kaggle/input/europeanpatentofficev2/en-up-guidelines-2025-pre-publication.pdf


In [3]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install python-docx
!pip install -qU "langchain[mistralai]"
!pip install -qU langchain-mistralai
!pip install -qU langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 

In [17]:
import re
import getpass
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain import hub
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain.schema import Document as Doc_langshain
from langchain.chat_models import init_chat_model
from langchain_mistralai import MistralAIEmbeddings
from langchain_chroma import Chroma

In [18]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["LANGSMITH_TRACING"] = "false"
os.environ["MISTRAL_API_KEY"]   =  user_secrets.get_secret("MISTRAL_API_KEY")

In [19]:
def split_doc_by_question(text):
    """
    Découpe un document en sections à partir du mot-clé 'Question' suivi d'un numéro.
    Retourne une liste de sections (chaque question avec ses réponses et explications).
    """
    # Regex qui capte chaque bloc depuis 'Question X' jusqu'à la question suivante ou la fin
    pattern = re.compile(r'(Question\s*\d+.*?)(?=Question\s*\d+|\Z)', re.DOTALL | re.IGNORECASE)
    matches = pattern.findall(text)

    return [m.strip() for m in matches]

from docx import Document

# Charger le document Word
doc = Document("/kaggle/input/qst-and-answer-oeb/Questions Sup OEB.docx")

# Extraire tout le texte en un seul string
full_text = "\n".join([para.text for para in doc.paragraphs])

# Découper en blocs
questions = split_doc_by_question(full_text)

# Convertir les questions en objets Document
questions_as_documents = [
    Doc_langshain(page_content=question, metadata={"source": "questions"})
    for question in questions
]

In [20]:
llm = init_chat_model("mistral-large-latest", model_provider="mistralai")
embeddings = MistralAIEmbeddings(model="mistral-embed")
vector_store = Chroma(embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [21]:
# Ajouter les documents à vector_store
_ = vector_store.add_documents(documents=questions_as_documents)

In [22]:
# Définir le prompt pour la réponse à la question
# LangChain dispose d'un "hub" (un dépôt) où tu peux récupérer des modèles préétablis, 
# des prompts, ou d'autres ressources partagées.
prompt = hub.pull("rlm/rag-prompt")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [23]:
# Définir l'état de l'application avec un dictionnaire typé
class State(TypedDict):
    question: str  # La question posée par l'utilisateur
    context: List[Document]  # Contexte sous forme de liste de documents récupérés
    answer: str # La réponse générée pour la question

In [24]:
def retrieve(state: State):
    # Recherche de documents similaires à la question dans la base de données
    retrieved_docs_with_scores = vector_store.similarity_search_with_score(state["question"], k=3)
    
    # Extraire les documents récupérés
    retrieved_docs = [doc for doc, score in retrieved_docs_with_scores]
    
    # Extraire les scores des documents récupérés
    # distances = [score for doc, score in retrieved_docs_with_scores]
    #print("Distances des documents récupérés :", distances)  # Affichage optionnel des distances pour vérification
    
    return {"context": retrieved_docs}

In [25]:
# Fonction pour générer la réponse à la question en fonction du contexte
def generate(state: State):
    # Joindre le contenu des documents du contexte pour former un bloc de texte
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    
    # Générer un message à partir du prompt avec la question et le contexte
    messages = prompt.invoke({"question": state["question"], "context": docs_content})

    # Invoker le modèle de langage pour générer la réponse
    response = llm.invoke(messages)
    return {"answer": response.content}

In [26]:
# Compiler l'application et la tester
graph_builder = StateGraph(State).add_sequence([retrieve, generate])  # Ajouter les étapes à l'application
graph_builder.add_edge(START, "retrieve")  # Définir la première étape du graph comme "retrieve"
graph = graph_builder.compile() # Compiler le graph pour l'exécution

In [29]:
response = graph.invoke({
    "question": 
                        
    """An individual applicant from Spain filed a European patent application in Spanish and requested examination in Spanish.
     The examination and designation fees are due by November 7, 2023. The applicant asks if they are eligible for a 20% reduction in the examination fee. 
     What is the correct answer?
    """                                            
    })
print(response["answer"])

The applicant is entitled to a reduction in the examination fee, but the reduction is not 20%. The correct reduction is 30% as per Article 14(1) RFees. This is because the applicant is a natural person and filed the application and requested examination in Spanish, fulfilling the requirements for the fee reduction.


# NGROK : 
# Sur Kaggle Notebooks (ou tout autre environnement cloud type Google Colab), tu n’as pas d’accès direct aux ports de la machine, il faudra une passerelle.

In [30]:
%%writefile app.py
import os
import tempfile
import streamlit as st
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_mistralai import MistralAIEmbeddings
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from docx import Document
import re

# Configuration pour Kaggle
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["LANGSMITH_TRACING"] = "false"
os.environ["MISTRAL_API_KEY"]   =  user_secrets.get_secret("MISTRAL_API_KEY")

# Interface Streamlit
st.set_page_config(page_title="Exemple RAG", layout="centered")

st.title("📄 RAG Assistant")
st.markdown("### Posez des questions sur vos documents PDF!")

# Modèles avec cache
@st.cache_resource
def load_models():
    return (
        init_chat_model("mistral-large-latest", model_provider="mistralai"),
        MistralAIEmbeddings(model="mistral-embed"),
        Chroma(embedding_function=MistralAIEmbeddings(model="mistral-embed"))
    )

llm, embeddings, vector_store = load_models()
        
with st.spinner("Analyse ..."):
    # Charger le document Word
    doc = Document("/kaggle/input/qst-and-answer-oeb/Questions Sup OEB.docx")
    
    # Extraire tout le texte en un seul string
    full_text = "\n".join([para.text for para in doc.paragraphs])
    pattern = re.compile(r'(Question\s*\d+.*?)(?=Question\s*\d+|\Z)', re.DOTALL | re.IGNORECASE)
    matches = pattern.findall(full_text)
    splits =  [m.strip() for m in matches]
    
    from langchain.schema import Document

    splits = [
        Document(page_content=splt, metadata={"source": "questions"})
        for splt in splits
    ]

    vector_store = Chroma.from_documents(
        splits, 
        embeddings,
        persist_directory="/kaggle/working/chroma_db"
    )
    st.session_state.vector_store = vector_store
    st.success("RAG prêt!")

# Interface de question
if "vector_store" in st.session_state:
    question = st.text_input("Votre question :")
    
    if question:
        with st.spinner("Recherche..."):
            #docs = st.session_state.vector_store.similarity_search(question, k=1)
            #context = "\n\n".join(d.page_content for d in docs)
            context = st.session_state.vector_store.similarity_search(question)
            
            response = llm.invoke(
                hub.pull("rlm/rag-prompt").format(
                    question=question,
                    context=context
                )
            )
            
            st.markdown(f"**Réponse :**\n{response.content}")

Writing app.py


In [ ]:
!pip install -q streamlit langchain-text-splitters langchain-community langchain-chroma langchain-mistralai pypdf nest-asyncio pyngrok

import subprocess
from pyngrok import ngrok

# Configuration de Ngrok
AUTH_TOKEN =    user_secrets.get_secret("AUTH_TOKEN")# Remplace par ta clé Ngrok
ngrok.set_auth_token(AUTH_TOKEN)
# Configuration du tunnel
public_url = ngrok.connect(8501).public_url
print(f"Accès à l'application : {public_url}")

# Lancement de Streamlit en arrière-plan
!streamlit run app.py --server.port 8501 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 71.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.9 MB/s eta 0:00:00:00:0100:01
Accès à l'application : https://ea96-104-198-6-8.ngrok-free.app                                     



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://104.198.6.8:8501

/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
